# Part 1: Prepare the development environment -- only need to run once.

In [1]:
# import the required packages

from collections import defaultdict
import os
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import tarfile
import time
import torch
from torch.autograd import Variable

In [2]:
# functions for processing the data (i.e. extract interaction and affinity data from zipped files)
# the output is train/test csv files

def extract_data(measure, file):
    data = pd.read_csv(file, sep='\t', dtype=str)
    #print(data.columns)
    data = data[['Ligand SMILES', 'BindingDB Target Chain  Sequence', 'pKi_[M]', 'pIC50_[M]', 'pKd_[M]', 'pEC50_[M]']]
    data.columns = ['SMILES', 'Sequence', 'Ki', 'IC50', 'Kd', 'EC50']
    data = data[['SMILES', 'Sequence', measure]]
    if 'train' in file:
        data.to_csv('../data/csv/train_' + measure + '.csv', index=None, header=None)
    else:
        data.to_csv('../data/csv/test_' + measure + '.csv', index=None, header=None)

def affinity_data_prepare(dataset):
    data_dir = '../data/affinity/' + dataset
    if not os.path.isdir(data_dir):
        tar = tarfile.open(data_dir + '.tar.xz')
        os.mkdir(data_dir)
        for name in tar.getnames():
            tar.extract(name, '../data/affinity/')
        tar.close()

    extract_data(dataset, data_dir + '/train')
    extract_data(dataset, data_dir + '/test')

def interaction_data_prepare(dataset):
    if dataset == 'human':
        train = ('../data/interaction/' + dataset + '/train.txt')
        data = pd.read_csv(train, sep=',', dtype=str)
        data.columns = ['SMILES', 'Sequence', 'Target']
        data.to_csv('../data/csv/train_human.csv', index=None, header=None)
        
        test = ('../data/interaction/' + dataset + '/test.txt')
        data = pd.read_csv(test, sep=',', dtype=str)
        data.columns = ['SMILES', 'Sequence', 'Target']
        data.to_csv('../data/csv/test_human.csv', index=None, header=None)
    else:
        train = ('../data/interaction/' + dataset + '/train.txt')
        data = pd.read_csv(train, sep=',', dtype=str)
        data.columns = ['SMILES', 'Sequence', 'Target']
        data.to_csv('../data/csv/train_celegans.csv', index=None, header=None)
        
        test = ('../data/interaction/' + dataset + '/test.txt')
        data = pd.read_csv(test, sep=',', dtype=str)
        data.columns = ['SMILES', 'Sequence', 'Target']
        data.to_csv('../data/csv/test_celegans.csv', index=None, header=None)

In [3]:
%%time

for dataset in ['IC50', 'EC50', 'Ki', 'Kd']:
    affinity_data_prepare(dataset)
    
for dataset in ['human', 'celegans']:
    interaction_data_prepare(dataset)

Wall time: 32.7 s


### You should have 12 csv files.

<table>
<tbody>
  <tr>
    <td>test_celegans.csv</td>
    <td>test_IC50.csv</td>
    <td>train_celegans.csv</td>
    <td>train_IC50.csv</td>
  </tr>
  <tr>
    <td>test_EC50.csv</td>
    <td>test_Kd.csv</td>
    <td>train_EC50.csv</td>
    <td>train_Kd.csv</td>
  </tr>
  <tr>
    <td>test_human.csv</td>
    <td>test_Ki.csv</td>
    <td>train_human.csv</td>
    <td>train_Ki.csv</td>
  </tr>
</tbody>
</table>

# Part 2: Prepare the data.

In [4]:
# change the directory to that of the CSV files.

current = os.getcwd()
os.chdir('../data/csv/')

print(os.getcwd())

C:\Users\rabin badree\OneDrive\Documents\badreeRoziena\machineLearning\project\github\data\csv


In [5]:
data = pd.read_csv('test_EC50.csv', header=None)
primary = data.iloc[-1, :]  # three columns : smiles, sequence, interaction 
radius, ngram = 2, 3
mol = Chem.AddHs(Chem.MolFromSmiles(primary[0]))
adjacency = Chem.GetAdjacencyMatrix(mol)

print([a.GetSymbol() for a in mol.GetAtoms()])
print([str(b.GetBondType()) for b in mol.GetBonds()])
print([a.GetIdx() for a in mol.GetAromaticAtoms()])

#[int(i) for i in AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=64, useChirality=True).ToBitString()]

['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'N', 'C', 'O', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'Cl', 'N', 'O', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
['AROMATIC', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'SINGLE', 'DOUBLE', 'SINGLE', 'DOUBLE', 'SINGLE', 'SINGLE', 'DOUBLE', 'SINGLE', 'SINGLE', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'SINGLE', 'SINGLE', 'SINGLE', 'DOUBLE', 'SINGLE', 'AROMATIC', 'SINGLE', 'AROMATIC', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE']
[0, 1, 2, 3, 4, 5, 14, 15, 16, 17, 18, 19]


### Step 1: Generate the data for the compounds.

In [6]:
def create_atoms(mol, atom_dict):
# print(primary[0])    
# result: C1=CC=C(C(=C1)C2=C(C(=O)NC2=O)NC3=CC(=C(C=C3)O)Cl)[N+](=O)[O-]

  atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    
# print(atoms, len(atoms)) 
# result:['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'N', 'C', 'O', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'Cl', 'N', 'O', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'] 35
# print([a.GetIdx() for a in mol.GetAromaticAtoms()])
# result:[0, 1, 2, 3, 4, 5, 14, 15, 16, 17, 18, 19]

  for a in mol.GetAromaticAtoms():
    i = a.GetIdx()
    atoms[i] = (atoms[i], 'aromatic')
    
# print(atoms)
# result:[('C', 'aromatic'), ('C', 'aromatic'), ('C', 'aromatic'), ('C', 'aromatic'), ('C', 'aromatic'), ('C', 'aromatic'), 'C', 'C', 'C', 'O', 'N', 'C', 'O', 'N', ('C', 'aromatic'), ('C', 'aromatic'), ('C', 'aromatic'), ('C', 'aromatic'), ('C', 'aromatic'), ('C', 'aromatic'), 'O', 'Cl', 'N', 'O', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']

  atoms = [atom_dict[a] for a in atoms]
    
# print(atoms)
# result:[0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3, 1, 2, 3, 0, 0, 0, 0, 0, 0, 2, 4, 3, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

  return atoms

In [7]:
def create_ijbonddict(mol, bond_dict):
    
# print(primary[0])    
# result: C1=CC=C(C(=C1)C2=C(C(=O)NC2=O)NC3=CC(=C(C=C3)O)Cl)[N+](=O)[O-]

    i_jbond_dict = defaultdict(lambda: [])
    
#print([len(mol.GetBonds())])
# result: 37

    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
        
#print(i_jbond_dict)
# {0: [(1, 0), (5, 0), (25, 1)],
#  1: [(0, 0), (2, 0), (26, 1)],
#  2: [(1, 0), (3, 0), (27, 1)],
#  3: [(2, 0), (4, 0), (22, 1)],
#  4: [(3, 0), (5, 0), (6, 1)],
#  5: [(4, 0), (0, 0), (28, 1)],
#  6: [(4, 1), (7, 2), (11, 1)],
#  7: [(6, 2), (8, 1), (13, 1)],
#  8: [(7, 1), (9, 2), (10, 1)],
#  9: [(8, 2)],
#  10: [(8, 1), (11, 1), (29, 1)],
#  11: [(10, 1), (12, 2), (6, 1)],
#  12: [(11, 2)],
#  13: [(7, 1), (14, 1), (30, 1)],
#  14: [(13, 1), (15, 0), (19, 0)],
#  15: [(14, 0), (16, 0), (31, 1)],
#  16: [(15, 0), (17, 0), (21, 1)],
#  17: [(16, 0), (18, 0), (20, 1)],
#  18: [(17, 0), (19, 0), (32, 1)],
#  19: [(18, 0), (14, 0), (33, 1)],
#  20: [(17, 1), (34, 1)],
#  21: [(16, 1)],
#  22: [(3, 1), (23, 2), (24, 1)],
#  23: [(22, 2)],
#  24: [(22, 1)],
#  25: [(0, 1)],
#  26: [(1, 1)],
#  27: [(2, 1)],
#  28: [(5, 1)],
#  29: [(10, 1)],
#  30: [(13, 1)],
#  31: [(15, 1)],
#  32: [(18, 1)],
#  33: [(19, 1)],
#  34: [(20, 1)]})

# find the atoms that do not have bonds with other atoms 

    isolate_atoms = set(range(mol.GetNumAtoms())) - set(i_jbond_dict.keys())
    bond = bond_dict['nan']
    for a in isolate_atoms:
        i_jbond_dict[a].append((a, bond))

    return i_jbond_dict

In [8]:
def atom_features(atoms, i_jbond_dict, radius, edge_dict, fingerprint_dict):    
    if (len(atoms) == 1) or (radius == 0):
        fingerprints = [fingerprint_dict[a] for a in atoms]
    
    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict
        for _ in range(radius):
            fingerprints = []
# the first radius:
# i_jedge_dict has 35 items. The following loop converts the atom position number(from 0 to 34), into the atoms_type_dict number(totally 5 types).
# shrink the previous 35 atoms with edges, into 18 types of (atoms,edges)
# fingerprint = (atom_type_dict, tuple(sorted(neighbors)))    
# neighbor = (atom_type_dict, bond_type_dict)
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                fingerprints.append(fingerprint_dict[fingerprint])  # after converting into atom_type, many fingerprint are with the same value
# print([str(b.GetBondType()) for b in mol.GetBonds()])
# ['AROMATIC', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'SINGLE', 'DOUBLE', 'SINGLE', 'DOUBLE', 'SINGLE', 'SINGLE', 'DOUBLE', 'SINGLE', 'SINGLE', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'AROMATIC', 'SINGLE', 'SINGLE', 'SINGLE', 'DOUBLE', 'SINGLE', 'AROMATIC', 'SINGLE', 'AROMATIC', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE']
#print(fingerprints, len(fingerprints))
# the first radius of fingerprints: 
#[0, 0, 0, 1, 2, 0, 3, 4, 5, 6, 7, 5, 6, 8, 1, 0, 9, 10, 0, 0, 11, 12, 13, 14, 15, 16, 16, 16, 16, 17, 17, 16, 16, 16, 18] ---len:35
# the second radius of fingerprints: 
#[0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 8, 11, 12, 13, 14, 15, 16, 1, 17, 18, 19, 20, 21, 22, 22, 22, 22, 23, 24, 22, 22, 22, 25]--with atom type and edge type

# update the i_jedge_dict by disgarding the duplicated edges for both directions and only keeping one undirected edge     
# upgrade the edge type, from the original edge type, to combine both nodes' type and edge type      
            nodes = fingerprints
            _i_jedge_dict = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    edge = edge_dict[(both_side, edge)]
                    _i_jedge_dict[i].append((j, edge))
            #print(len(edge_dict))
            i_jedge_dict = _i_jedge_dict

    return np.array(fingerprints)


### Step 2: Generate the adjacency.

In [9]:
def create_adjacency(mol):
    adjacency = Chem.GetAdjacencyMatrix(mol)
    adjacency = np.array(adjacency)
    adjacency += np.eye(adjacency.shape[0], dtype=int)
    return adjacency

### Step 3: Generate the fingerprints.

In [10]:
def get_fingerprints(mol):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=64, useChirality=True)
    return fp.ToBitString()

### Step 4: Generate the protein sequence.

In [11]:
def split_sequence(sequence, ngram, word_dict):
    sequence = '-' + sequence + '='
    words = [word_dict[sequence[i:i+ngram]]
             for i in range(len(sequence)-ngram+1)]
    return np.array(words)

# experiment:
#split_sequence(primary[1], ngram)

### Step 5: Generate the compounds, adjacencies, fps, proteins, interactions with the input data.

In [12]:
def tokenize_input_data(input_file, dataset, task, name, radius, ngram): 
# name = train/test, task = interaction/affinity
    data = pd.read_csv(input_file, header=None)
    data = data[0:1000]
    word_dict = defaultdict(lambda: len(word_dict))
    fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
    atom_dict = defaultdict(lambda: len(atom_dict))
    edge_dict = defaultdict(lambda: len(edge_dict))
    bond_dict = defaultdict(lambda: len(bond_dict))
    X = []
    y = []
    compounds, adjacencies, fps, proteins, interactions = [], [], [], [], []

    for i in range(len(data)):
        smiles, sequence, interaction = data.iloc[i, :]

        mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
        atoms = create_atoms(mol, atom_dict)
        i_jbond_dict = create_ijbonddict(mol, bond_dict)
        compounds.append(atom_features(atoms, i_jbond_dict, radius, edge_dict, fingerprint_dict))
        adjacencies.append(create_adjacency(mol))
        fps.append(get_fingerprints(mol))
        proteins.append(split_sequence(sequence, ngram, word_dict))
        interactions.append(np.array([float(interaction)]))
    return [compounds, adjacencies, fps, proteins, interactions], fingerprint_dict, word_dict


"""
    output_path = os.path.join('../datasets', task, dataset, name)
    os.makedirs(output_path, exist_ok=True)
    np.save(os.path.join(output_path, 'compounds'), compounds)
    np.save(os.path.join(output_path, 'adjacencies'), adjacencies)
    np.save(os.path.join(output_path, 'fingerprint'), fps)
    np.save(os.path.join(output_path, 'proteins'), proteins)
    np.save(os.path.join(output_path, 'interactions'), interactions)

    with open(os.path.join('../datasets', task, dataset, name, 'atom_dict'), 'wb') as f:
        pickle.dump(dict(fingerprint_dict), f)
    with open(os.path.join('../datasets', task, dataset, name, 'amino_dict'), 'wb') as f:
        pickle.dump(dict(word_dict), f)
"""
# experiment with writing the corresponding files:
#tokenize_input_data('test_EC50.csv', 'EC50', 'affinity', 'test', 2, 3)
#tokenize_input_data('train_EC50.csv', 'EC50', 'affinity', 'train', 2, 3)
#tokenize_input_data('test_IC50.csv', 'IC50', 'affinity', 'test', 2, 3)
#tokenize_input_data('train_IC50.csv', 'IC50', 'affinity', 'train', 2, 3)
#tokenize_input_data('test_Kd.csv', 'Kd', 'affinity', 'test', 2, 3)
#tokenize_input_data('train_Kd.csv', 'Kd', 'affinity', 'train', 2, 3)
#tokenize_input_data('test_Ki.csv', 'Ki', 'affinity', 'test', 2, 3)
#tokenize_input_data('train_Ki.csv', 'Ki', 'affinity', 'train', 2, 3)
#tokenize_input_data('test_celegans.csv', 'celegans', 'interaction', 'test', 2, 3)
#tokenize_input_data('train_celegans.csv', 'celegans', 'interaction', 'train', 2, 3)
#tokenize_input_data('test_human.csv', 'human', 'interaction', 'test', 2, 3)
#tokenize_input_data('train_human.csv', 'human', 'interaction', 'train', 2, 3)

"\n    output_path = os.path.join('../datasets', task, dataset, name)\n    os.makedirs(output_path, exist_ok=True)\n    np.save(os.path.join(output_path, 'compounds'), compounds)\n    np.save(os.path.join(output_path, 'adjacencies'), adjacencies)\n    np.save(os.path.join(output_path, 'fingerprint'), fps)\n    np.save(os.path.join(output_path, 'proteins'), proteins)\n    np.save(os.path.join(output_path, 'interactions'), interactions)\n\n    with open(os.path.join('../datasets', task, dataset, name, 'atom_dict'), 'wb') as f:\n        pickle.dump(dict(fingerprint_dict), f)\n    with open(os.path.join('../datasets', task, dataset, name, 'amino_dict'), 'wb') as f:\n        pickle.dump(dict(word_dict), f)\n"

# Part 4: Train the models.

In [13]:
affinity_train_data = ['train_EC50.csv', 'train_IC50.csv', 'train_Kd.csv', 'train_Ki.csv']
affinity_test_data = ['test_EC50.csv', 'test_IC50.csv', 'test_Kd.csv', 'test_Ki.csv']
interaction_train_data = ['train_celegans.csv', 'train_human.csv']
interaction_test_data = ['test_celegans.csv', 'test_human.csv']

# data = [compounds, adjacencies, fps, proteins, interactions]
data, atom_dict, amino_dict = tokenize_input_data(affinity_train_data[0], 'EC50', 'affinity', 'test', 2, 3)

In [14]:
def batch_pad(arr):
    N = max([a.shape[0] for a in arr])
    if arr[0].ndim == 1:
        new_arr = np.zeros((len(arr), N))
        new_arr_mask = np.zeros((len(arr), N))
        for i, a in enumerate(arr):
            n = a.shape[0]
            new_arr[i, :n] = a + 1
            new_arr_mask[i, :n] = 1
        return new_arr, new_arr_mask

    elif arr[0].ndim == 2:
        new_arr = np.zeros((len(arr), N, N))
        new_arr_mask = np.zeros((len(arr), N, N))
        for i, a in enumerate(arr):
            n = a.shape[0]
            new_arr[i, :n, :n] = a
            new_arr_mask[i, :n, :n] = 1
        return new_arr, new_arr_mask


def fps2number(arr):
    new_arr = np.zeros((len(arr), 64))
    for i, a in enumerate(arr):
        new_arr[i, :] = np.array(list(a), dtype=int)
    return new_arr


def batch2tensor(batch_data, device):
    atoms_pad, atoms_mask = batch_pad(batch_data[0])
    adjacencies_pad, _ = batch_pad(batch_data[1])
    fps = fps2number(batch_data[2])
    amino_pad, amino_mask = batch_pad(batch_data[3])

    atoms_pad = Variable(torch.LongTensor(atoms_pad)).to(device)
    atoms_mask = Variable(torch.FloatTensor(atoms_mask)).to(device)
    adjacencies_pad = Variable(torch.LongTensor(adjacencies_pad)).to(device)
    fps = Variable(torch.FloatTensor(fps)).to(device)
    amino_pad = Variable(torch.LongTensor(amino_pad)).to(device)
    amino_mask = Variable(torch.FloatTensor(amino_mask)).to(device)

    label = torch.FloatTensor(batch_data[4]).to(device)

    return atoms_pad, atoms_mask, adjacencies_pad, fps, amino_pad, amino_mask, label

device = torch.device('cpu')
atoms_pad, atoms_mask, adjacencies_pad, batch_fps, amino_pad, amino_mask, label = batch2tensor([data[0],data[1],data[2],data[3],data[4]], device)

print(atoms_pad.size())        
print(torch.flatten(adjacencies_pad, start_dim=1).size())    
print(batch_fps.size())    
print(amino_pad.size())   
print(label.size())   

torch.Size([1000, 107])
torch.Size([1000, 11449])
torch.Size([1000, 64])
torch.Size([1000, 1434])
torch.Size([1000, 1])


C:\Users\rabin badree\AppData\Local\Temp\ipykernel_2544\1313135390.py:42: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  label = torch.FloatTensor(batch_data[4]).to(device)


In [15]:
X = pd.DataFrame(torch.cat((atoms_pad,torch.flatten(adjacencies_pad, start_dim=1),batch_fps, amino_pad), -1).numpy())
y = pd.DataFrame(label.numpy())

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 13054 entries, 0 to 13053
dtypes: float32(13054)
memory usage: 49.8 MB


In [17]:
X.describe()

,0,1,2,3,4,5,6,7,8,9,...,13044,13045,13046,13047,13048,13049,13050,13051,13052,13053
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,361.795990,775.789001,702.020996,747.564026,726.793030,784.604980,799.432007,791.070984,741.171997,711.294983,...,24.719999,60.950001,33.540001,62.310001,22.740000,36.549999,23.510000,15.690000,14.400000,62.320000
std,370.343536,585.078308,550.717041,574.165894,599.627686,633.068665,620.204468,619.466553,619.741272,587.457275,...,246.084183,606.748047,333.886230,620.286011,226.373108,363.849884,234.038422,156.191589,143.350082,620.385498
min,14.000000,14.000000,15.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,138.000000,301.000000,247.000000,248.000000,223.000000,223.000000,242.000000,228.000000,207.000000,183.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,221.000000,591.000000,574.000000,595.500000,588.000000,587.000000,642.000000,640.000000,636.000000,629.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,418.500000,1363.500000,970.250000,1190.000000,1130.000000,1369.000000,1389.000000,1389.000000,1216.000000,1147.750000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2044.000000,2374.000000,2375.000000,2376.000000,2377.000000,2336.000000,2378.000000,2369.000000,2377.000000,2371.000000,...,2472.000000,6095.000000,3354.000000,6231.000000,2274.000000,3655.000000,2351.000000,1569.000000,1440.000000,6232.000000


In [18]:
print("Dimension of the features data: ", X.shape, " No. of Rows: %d" % X.shape[0], " No. of Columns: %d" % X.shape[1])
print("Dimension of the target data: ",y.shape)

Dimension of the features data:  (1000, 13054)  No. of Rows: 1000  No. of Columns: 13054
Dimension of the target data:  (1000, 1)


In [19]:
sum(X.isna().any())

0

In [20]:
y_train.head()

,0
687,5.408936
500,5.164310
332,7.102373
979,4.004365
817,7.397940


In [21]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.5)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
mean_squared_error(y_test, y_pred)
 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


1.2981262